---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [1]:
S = 60                      # initial price of underlying
K = 50                      # strike price
r = 0.01                    # risk-free rate per period
q = 0.01                    # dividend yield
sigma = 0.40                # volatility per year
T = 1                       # years to maturity
N = 100                     # number of periods

# FUNCTIONS

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import plotly.graph_objects as go


def callBS(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return np.exp(-q * T) * s * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)


def putBS(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return np.exp(-r * T) * K * norm.cdf(-d2) - np.exp(-q * T) * s * norm.cdf(-d1)

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)


def callEuropean(S, K, T, sigma, r, q, N=100):
    def f(s):
        s = s if s != 0 else 1.0e-6  # to allow 0 in the stock price grid when we use it
        dt = T / N
        up = np.exp(sigma * np.sqrt(dt))
        down = 1 / up
        prob = (np.exp((r - q) * dt) - down) / (up - down)
        discount = np.exp(-r * dt)
        v = np.zeros(N + 1)
        x = s * up ** N
        v[0] = np.maximum(x - K, 0)

        # v will become the last column of the df
        for i in range(1, N + 1):
            x *= down * down
            v[i] = np.maximum(x - K, 0)

        # x now is the lowest price

        # for the previous N columns
        for n in range(N - 1, -1, -1):
            x = s * up ** n  # x now is the highest price of the n-th column
            v[0] = discount * (prob * v[0] + (1 - prob) * v[1])
            for i in range(1, n + 1):
                x *= down * down
                v[i] = discount * (prob * v[i] + (1 - prob) * v[i + 1])
            # v now is the previous column
        return v[0]

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)


def putEuropean(S, K, T, sigma, r, q, N=100):
    def f(s):
        s = s if s != 0 else 1.0e-6
        dt = T / N
        up = np.exp(sigma * np.sqrt(dt))
        down = 1 / up
        prob = (np.exp((r - q) * dt) - down) / (up - down)
        discount = np.exp(-r * dt)
        v = np.zeros(N + 1)
        x = s * up ** N
        v[0] = np.maximum(K - x, 0)
        for i in range(1, N + 1):
            x *= down * down
            v[i] = np.maximum(K - x, 0)
        for n in range(N - 1, -1, -1):
            x = s * up ** n
            v[0] = discount * (prob * v[0] + (1 - prob) * v[1])
            for i in range(1, n + 1):
                x *= down * down
                v[i] = discount * (prob * v[i] + (1 - prob) * v[i + 1])
        return v[0]

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)

# CALCULATIONS

In [3]:
# simulate even numbers of depths only
maxN = int(N/2)

# grid of time steps
a = np.linspace(1, maxN, maxN) * 2

##### call calculations
# binomial model
call = np.zeros(maxN)
for i in range(maxN):
    call[i] = callEuropean(S, K, T, sigma, r, q, int(a[i]))

# Black-Scholes model
call_bs = callBS(S, K, T, sigma, r, q)


##### put calculations
# binomial model
put = np.zeros(maxN)
for i in range(maxN):
    put[i] = putEuropean(S, K, T, sigma, r, q, int(a[i]))

# Black-Scholes model
put_bs = putBS(S, K, T, sigma, r, q)

# FIGURE: EUROPEAN CALL

In [4]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=a, 
        y=call, 
        mode="markers",
        marker=dict(size=10),
        name="Binomial Value",
        hovertemplate="Binomial value = $%{y:0.2f}<extra></extra>")
    )
fig.add_trace(
    go.Scatter(
        x = np.linspace(0, 2*maxN, 100),
        y= [call_bs] * 100,
        name = "Black-Scholes",
        hovertemplate="Black-Scholes value = $%{y:0.2f}<extra></extra>"
    )
)
fig.update_layout(
    xaxis_title = "Number of Time Steps in Binomial Model",
    yaxis_title = "Option Value",
    yaxis_tickprefix="$", 
    yaxis_tickformat=",.2f",
    template="plotly_white",    
    title={
        "text": "European Call",
        "y": 0.96,
        "x": 0.2,
        "xanchor": "center",
        "yanchor": "bottom",
    },
    legend=dict(
        yanchor="top",
        y=0.99, 
        xanchor="right", 
        x=0.90
    )
)
fig.show()


# FIGURE: EUROPEAN PUT

In [5]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=a, 
        y=put, 
        mode="markers",
        marker=dict(size=10),
        name="Binomial Value",
        hovertemplate="Binomial value = $%{y:0.2f}<extra></extra>")
    )
fig.add_trace(
    go.Scatter(
        x = np.linspace(0, 2*maxN, 100),
        y= [put_bs] * 100,
        name = "Black-Scholes",
        hovertemplate="Black-Scholes value = $%{y:0.2f}<extra></extra>"
    )
)
fig.update_layout(
    xaxis_title = "Number of Time Steps in Binomial Model",
    yaxis_title = "Option Value",
    yaxis_tickprefix="$", 
    yaxis_tickformat=",.2f",
    template="plotly_white",    
    title={
        "text": "European Put",
        "y": 0.96,
        "x": 0.2,
        "xanchor": "center",
        "yanchor": "bottom",
    },
    legend=dict(
        yanchor="top",
        y=0.99, 
        xanchor="right", 
        x=0.90
    )
)
fig.show()